In [1]:
import pandas as pd
pd.set_option("display.max_colwidth", 200)
import glob
import os
import pickle

### System-level data

In [2]:
wmt17_no_metadata = pickle.load(open('../data/pickles/wmt17-sys_level-all.pkl', 'rb'))
wmt17_no_metadata.count()

system         203012
sid            203012
raw_score      203012
score          203012
N              203012
lp             203012
reference      203012
source         203012
output         203012
system_full    203012
dtype: int64

In [3]:
# fix spaces in ad-good-raw file
import gzip
write = open('./data/ad-raw.csv', 'wt')
read = gzip.open('./data/newstest2017-system-level-human/anon-proc-hits-sys-combined/analysis/ad-good-raw-redup.csv.gz', 'rt')

# DUPLICATES all human judgments for systemA+systemB
for line in read:
    if '+' not in line:
        write.write(line.replace(' ', '\t'))
    else:
        line = line.split('\t')
        system = line[6]
        for i in system.split('+'):
            l = '\t'.join(line[0:6]) + '\t' + i + '\t' + '\t'.join(line[7:])
            write.write(l)
write.close()

In [4]:
df = pd.read_csv('./data/ad-raw.csv', sep='\t')
df.count()

/nas/home/jwei/anaconda3/envs/notebook/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3062: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


HITId         375325
WorkerId      375325
Input.src     375325
Input.trg     375325
Input.item    375325
hit           375325
sys_id        375325
rid           249632
type          375325
sid           375325
score         375325
time          375325
dtype: int64

In [5]:
df['lp'] = df.apply(lambda x: '%s-%s' % (x['Input.src'], x['Input.trg']), axis=1)
df.replace('ROCMT.5167', 'ROCMT.5183', inplace=True)
df.replace('CASICT-cons.5144', 'CASICT-DCU-NMT.5144', inplace=True)
df.replace('CASICT-cons.5157', 'CASICT-DCU-NMT.5157', inplace=True)
df.replace('fbk-nmt-combination.4870', 'FBK.4870', inplace=True)
df.head(1)

,HITId,WorkerId,Input.src,Input.trg,Input.item,hit,sys_id,rid,type,sid,score,time,lp
0,374UMBUHN6UORKZ155BBXHOJ7ALCTX,M0106,cs,en,ad,0,online-B.0,0.0,SYSTEM,905,49,3062.0,cs-en


In [6]:
wmt17_metadata = df.merge(wmt17_no_metadata[['system_full', 'sid', 'lp', 'reference', 'source', 'output']],
         left_on=['lp', 'sys_id', 'sid'],
         right_on=['lp', 'system_full', 'sid'],
         how='left'
        )
wmt17_metadata.count()

HITId          375325
WorkerId       375325
Input.src      375325
Input.trg      375325
Input.item     375325
hit            375325
sys_id         375325
rid            249632
type           375325
sid            375325
score          375325
time           375325
lp             375325
system_full    362523
reference      362523
source         362523
output         362523
dtype: int64

In [7]:
wmt17_no_metadata[wmt17_no_metadata.system.str.contains('tuning')].system_full.unique()

array(['tuning-task-afrl_4gb.sgm.0', 'tuning-task-ufal_4gb.sgm.0',
       'tuning-task-afrl_8gb.sgm.0', 'tuning-task-baseline_8gb.sgm.0',
       'tuning-task-denisov_4gb.sgm.0', 'tuning-task-ufal_8gb.sgm.0'],
      dtype=object)

In [8]:
wmt17_metadata[wmt17_metadata.system_full.isna() & (wmt17_metadata.type == 'SYSTEM')].sys_id.unique()

array(['tuning-task-baseline_4gb.sgm.0'], dtype=object)

In [9]:
print(len(wmt17_metadata.groupby(['lp', 'sid', 'sys_id'], as_index=False).count()))
print(len(wmt17_no_metadata))

212989
203012


In [10]:
wmt17_metadata = wmt17_metadata[['lp', 'HITId', 'WorkerId', 'score', 'time', 'sys_id', 'type', 'sid', 'reference', 'source', 'output']]

In [11]:
pickle.dump(wmt17_metadata, open('pickles/wmt17_sys_metadata.pkl', 'wb'))

### Segment-level data

In [12]:
wmt17_no_metadata = pickle.load(open('../data/pickles/wmt17-seg_level-agg.pkl', 'rb'))
wmt17_no_metadata.count()

lp           5040
system       5040
sid          5040
output       5040
source       5040
reference    5040
score        5040
sentBLEU     4421
dtype: int64

In [13]:
# still missing en-zh 
# and en-ru

In [14]:
df = pd.read_csv('./data/newstest2017-segment-level-human/anon-proc-hits-seg-en/analysis/ad-good-raw.csv.gz', sep='\t')
df.groupby(['Input.src', 'Input.trg', 'sys_id', 'sid'], as_index=False).count()

,Input.src,Input.trg,sys_id,sid,HITId,WorkerId,Input.item,hit,rid,type,score,time
0,cs,en,PJATK.4760,37,17,17,17,17,17,17,17,17
1,cs,en,PJATK.4760,63,30,30,30,30,30,30,30,30
2,cs,en,PJATK.4760,65,16,16,16,16,16,16,16,16
3,cs,en,PJATK.4760,86,30,30,30,30,30,30,30,30
4,cs,en,PJATK.4760,109,15,15,15,15,15,15,15,15
...,...,...,...,...,...,...,...,...,...,...,...,...
3915,zh,en,xmunmt.5160+NRC.5172+uedin-nmt.5112,262,15,15,15,15,15,15,15,15
3916,zh,en,xmunmt.5160+afrl-mitll-opennmt.5109+uedin-nmt.5112,1414,30,30,30,30,30,30,30,30
3917,zh,en,xmunmt.5160+online-B.0,33,17,17,17,17,17,17,17,17
3918,zh,en,xmunmt.5160+uedin-nmt.5112,243,34,34,34,34,34,34,34,34


In [15]:
df['lp'] = df.apply(lambda x: '%s-%s' % (x['Input.src'], x['Input.trg']), axis=1)
df['system'] = df.sys_id.apply(lambda x: x.split('+')[0])
df.replace('ROCMT.5167', 'ROCMT.5183', inplace=True)
df.replace('CASICT-cons.5144', 'CASICT-DCU-NMT.5144', inplace=True)
df.head(1)

,HITId,WorkerId,Input.src,Input.trg,Input.item,hit,sys_id,rid,type,sid,score,time,lp,system
0,341YLJU21J4LKOROE0N17XDOKBZI22,E0829,cs,en,ad,2,online-A.0,0,SYSTEM,446,99,2086,cs-en,online-A.0


In [16]:
wmt17_metadata = df.merge(wmt17_no_metadata[['system', 'sid', 'lp', 'reference', 'source', 'output']],
         left_on=['lp', 'system', 'sid'],
         right_on=['lp', 'system', 'sid'],
         how='left'
        )
wmt17_metadata.count()

HITId         88800
WorkerId      88800
Input.src     88800
Input.trg     88800
Input.item    88800
hit           88800
sys_id        88800
rid           88800
type          88800
sid           88800
score         88800
time          88800
lp            88800
system        88800
reference     88800
source        88800
output        88800
dtype: int64

In [17]:
wmt17_metadata = wmt17_metadata[['lp', 'HITId', 'WorkerId', 'score', 'time', 'system', 'type', 'sid', 'reference', 'source', 'output']]

In [18]:
pickle.dump(wmt17_metadata, open('pickles/wmt17_seg_metadata.pkl', 'wb'))